# 🔬 StrandWeaver — ErrorSmith Training (Colab GPU)

Train **1 XGBoost multiclass classifier** for per-base sequencing error prediction.
Training uses **chemistry-specific** (flow cell / machine / chemistry) BAMs aligned to **HG002v1.0.1** (Q100 ground truth) and/or **CHM13v2.0**.

| Class | Label | Description |
|-------|-------|-------------|
| 0 | `correct` | Base matches reference |
| 1 | `substitution` | Wrong base |
| 2 | `insertion` | Extra base in read |
| 3 | `deletion` | Missing base in read |
| 4 | `homopolymer_error` | Ins/del in homopolymer context |

**Data sources:**
- **HG002v1.0.1** — Q100 T2T assembly, 10 BAMs across 10 chemistry codes (see config cell)
- **CHM13v2.0** — Original training reference (legacy, 6 chemistries)

| Code | Chemistry | Platform | HG002 BAMs |
|------|-----------|----------|------------|
| 0 | `pacbio_hifi_sequel2` | PacBio HiFi (Sequel II/IIe) | — |
| 1 | `ont_lsk110_r941` | ONT LSK-110, R9.4.1 | — |
| 2 | `ont_ulk001_r941` | ONT ULK-001, R9.4.1 (ultra long) | — |
| 3 | `ont_lsk114_r1041` | ONT LSK-114, R10.4.1 (standard ligation) | — |
| 4 | `ont_ulk114_r1041` | ONT ULK-114, R10.4.1 (standard ultra long) | ✓ |
| 5 | `illumina_hiseq2500` | Illumina HiSeq 2500 | ✓ 2×250 |
| 6 | `pacbio_onso` | PacBio Onso (short-read SBB, NOT HiFi) | ✓ NIST 2024Q1 |
| 7 | `element_aviti` | Element Aviti (standard / long) | ✓ Standard + Long |
| 8 | `element_ultraq` | Element UltraQ | ✓ NIST Jun 2024 |
| 9 | `pacbio_hifi_revio` | PacBio HiFi (Revio) | ✓ 3-cell |
| 10 | `ont_r1041_duplex` | ONT R10.4.1 Duplex | ✓ |
| 11 | `ont_ulk114_r1041_hiacc` | ONT UL R10.4.1 High-Accuracy (exp.) | ✓ |
| 12 | `ont_ulk114_r1041_dorado` | ONT UL R10.4.1 Dorado | ✓ |

### Setup
1. `Runtime → Change runtime type → T4 GPU` (or A100 if available)
2. Run cells top-to-bottom

### Data options
- **Option 1 (recommended):** Transfer HG002 BAMs to Google Drive via E2 rclone (see `Non_Main_Commit_Files/hg002_transfer/`)
- **Option 2:** Upload pre-aligned CHM13 BAMs to Google Drive `ErrorSmith_bams/`

> **K-Weaver training** is in a separate notebook: `KWeaver_Training_Colab.ipynb`

## 1. Environment Setup

In [ ]:
# ── Verify GPU ──────────────────────────────────────────────────────
import subprocess, sys, os

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=name,memory.total,driver_version', '--format=csv,noheader'],
    capture_output=True, text=True
)
if result.returncode == 0:
    print(f"✓ GPU detected: {result.stdout.strip()}")
    GPU_AVAILABLE = True
else:
    print("⚠ No GPU detected — XGBoost will use CPU (still fast, just slower HP search)")
    GPU_AVAILABLE = False

# Check RAM
import psutil
ram_gb = psutil.virtual_memory().total / 1e9
print(f"✓ RAM: {ram_gb:.1f} GB {'(High-RAM ✓)' if ram_gb > 20 else '(consider High-RAM runtime)'}")
print(f"✓ Python: {sys.version.split()[0]}")

In [ ]:
# ── Install dependencies ────────────────────────────────────────────
!pip install -q xgboost scikit-learn numpy pandas optuna pysam biopython

# Clone StrandWeaver (dev branch) and install — pull latest on re-runs
!git clone -b dev https://github.com/pgrady1322/strandweaver.git /content/strandweaver 2>/dev/null || \
    (cd /content/strandweaver && git pull origin dev)
!cd /content/strandweaver && pip install -e . -q

print("\n✓ Dependencies installed")

In [ ]:
# ── Mount Google Drive (for saving results) ─────────────────────────
from google.colab import drive
drive.mount('/content/drive')

# ── Directory layout ────────────────────────────────────────────────
GDRIVE_DIR        = '/content/drive/MyDrive/Colab_Training'
ERRORSMITH_OUT    = '/content/errorsmith_training'
MODELS_OUT        = '/content/trained_models'
GDRIVE_ERRORSMITH = os.path.join(GDRIVE_DIR, 'errorsmith_models')

for d in [GDRIVE_DIR, ERRORSMITH_OUT, MODELS_OUT]:
    os.makedirs(d, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Working dirs created")
print(f"  ErrorSmith output:  {ERRORSMITH_OUT}")
print(f"  Models output:      {MODELS_OUT}")
print(f"  Drive export:       {GDRIVE_DIR}")

In [ ]:
# ── Verify StrandWeaver imports ──────────────────────────────────────
sys.path.insert(0, '/content/strandweaver')
sys.path.insert(0, '/content/strandweaver/strandweaver/training/scripts/')

from strandweaver.io_utils import SeqRead

import xgboost as xgb
import numpy as np
import pandas as pd
import time
import logging
from pathlib import Path

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger('colab_training')

SEED = 42

# XGBoost device config
XGB_DEVICE = 'cuda' if GPU_AVAILABLE else 'cpu'
XGB_TREE = 'hist'  # gpu_hist is deprecated in XGBoost 2.x — 'hist' auto-uses GPU

print(f"✓ StrandWeaver imports OK")
print(f"✓ XGBoost {xgb.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ Pandas {pd.__version__}")

## 2. Configuration & Data Source

**Edit the cell below to match your setup.**

### Option 1: HG002v1.0.1 BAMs from Google Drive (recommended)

Transfer HG002 BAMs using the E2 rclone scripts in `Non_Main_Commit_Files/hg002_transfer/`.
After transfer, BAMs will be at:
```
My Drive/Colab_Training/ErrorSmith_bams/HG002/
├── illumina/             (HiSeq 2500, 2×250)           code 5
├── pacbio_hifi_revio/    (Revio HiFi)                  code 9
├── pacbio_onso/          (Onso short-read — NOT HiFi)   code 6
├── ont_lsk114_duplex/    (R10.4.1 Duplex)               code 10
├── ont_ulk114/           (UL R10 standard)              code 4
├── ont_ulk114_hiacc/     (UL R10 HiAcc experimental)    code 11
├── ont_ulk114_dorado/    (UL R10 Dorado basecaller)     code 12
├── element_aviti/        (Aviti Standard + Long)         code 7
├── element_ultraq/       (UltraQ)                       code 8
└── reference/            (HG002v1.0.1.fasta.gz)
```

### Option 2: CHM13 BAMs (legacy)

If using CHM13 data, upload to `ErrorSmith_bams/CHM13/` — the config cell detects both.

### Option 3: Upload FASTQs → align on Colab

Set `ERRORSMITH_MODE = 'fastq'` and provide FASTQs in `Colab_Training/fastq/`.

In [ ]:
# ══════════════════════════════════════════════════════════════════════
#  ERRORSMITH CONFIGURATION — edit these before running
# ══════════════════════════════════════════════════════════════════════

# ── Data source mode ────────────────────────────────────────────────
# 'bam'  : Use pre-aligned BAMs on Google Drive (recommended)
# 'fastq': Upload FASTQs to Google Drive, align on Colab
ERRORSMITH_MODE = 'bam'

# ── BAM directory (all BAMs live here) ──────────────────────────────
BAM_DIR = os.path.join(GDRIVE_DIR, 'ErrorSmith_bams')
HG002_BAM_DIR = os.path.join(BAM_DIR, 'HG002')
CHM13_BAM_DIR = os.path.join(BAM_DIR, 'CHM13')  # legacy, optional

# ══════════════════════════════════════════════════════════════════════
#  HG002v1.0.1 BAM MAP (Q100 ground truth — primary training data)
# ══════════════════════════════════════════════════════════════════════
# Transferred via Non_Main_Commit_Files/hg002_transfer/transfer_hg002_bams.sh
# Each chemistry key matches a CHEMISTRY_CODES entry (0–12).

HG002_BAM_MAP = {
    # Code 4: ONT ULK-114 R10.4.1 (ultra-long, standard basecaller)
    'ont_ulk114_r1041': [
        os.path.join(HG002_BAM_DIR, 'ont_ulk114', 'hg002v1.0_ont_r10_ul.pri.bam'),
    ],
    # Code 5: Illumina HiSeq 2500 (2×250)
    'illumina_hiseq2500': [
        os.path.join(HG002_BAM_DIR, 'illumina', 'hg002v1.0.1_illumina2x250_hg002.dedup.bam'),
    ],
    # Code 6: PacBio Onso (short-read SBB — NOT HiFi, entirely different)
    'pacbio_onso': [
        os.path.join(HG002_BAM_DIR, 'pacbio_onso', 'hg002v1.0.1_onso_hg002_NIST_2024Q1.dedup.bam'),
    ],
    # Code 7: Element Aviti (standard + long read)
    'element_aviti': [
        os.path.join(HG002_BAM_DIR, 'element_aviti', 'hg002v1.0.1_element_hg002_avitistd.dedup.bam'),
        os.path.join(HG002_BAM_DIR, 'element_aviti', 'hg002v1.0.1_hg002_element_avitilng.dedup.bam'),
    ],
    # Code 8: Element UltraQ
    'element_ultraq': [
        os.path.join(HG002_BAM_DIR, 'element_ultraq', 'hg002v1.0.1_element_hg002_ultraq_jun2024.1.dedup.bam'),
    ],
    # Code 9: PacBio HiFi Revio (distinct from Sequel II)
    'pacbio_hifi_revio': [
        os.path.join(HG002_BAM_DIR, 'pacbio_hifi_revio', 'hg002v1.0_hifi_revio_3cells.pri.bam'),
    ],
    # Code 10: ONT R10.4.1 Duplex basecalling
    'ont_r1041_duplex': [
        os.path.join(HG002_BAM_DIR, 'ont_lsk114_duplex', 'hg002v1.0_ont_r10_duplex.pri.bam'),
    ],
    # Code 11: ONT UL R10.4.1 High-Accuracy (experimental)
    'ont_ulk114_r1041_hiacc': [
        os.path.join(HG002_BAM_DIR, 'ont_ulk114_hiacc', 'all_pass.vhg002v1.bam'),
    ],
    # Code 12: ONT UL R10.4.1 Dorado basecaller
    'ont_ulk114_r1041_dorado': [
        os.path.join(HG002_BAM_DIR, 'ont_ulk114_dorado', 'hg002v1.0_ont_r10_ul_dorado.pri.bam'),
    ],
}

# ══════════════════════════════════════════════════════════════════════
#  CHM13 BAM MAP (legacy — optional, merged if present)
# ══════════════════════════════════════════════════════════════════════
CHM13_BAM_MAP = {
    'pacbio_hifi_sequel2': [
        os.path.join(CHM13_BAM_DIR, 'chm13_SRR11292120_HiFi.sorted.bam'),
        os.path.join(CHM13_BAM_DIR, 'chm13_SRR11292121_HiFi.sorted.bam'),
    ],
    'ont_lsk110_r941':     None,   # TODO: map LSK-110 R9.4.1 reads → CHM13
    'ont_ulk001_r941': [
        os.path.join(CHM13_BAM_DIR, 'chm13_SRR23513621_ONT_UL_R941.sorted.bam'),
    ],
    'ont_lsk114_r1041': [
        os.path.join(CHM13_BAM_DIR, 'chm13_SRR35645112_ONT_SL_R1041.sorted.bam'),
    ],
    'ont_ulk114_r1041':    None,   # covered by HG002
    'illumina_hiseq2500': [
        os.path.join(CHM13_BAM_DIR, 'chm13_SRR1997411_Illumina_PCR_Free.sorted.bam'),
    ],
}

# ── Merge BAM maps (HG002 takes priority) ──────────────────────────
BAM_MAP = {}
for chem in set(list(HG002_BAM_MAP.keys()) + list(CHM13_BAM_MAP.keys())):
    hg002_paths = HG002_BAM_MAP.get(chem) or []
    chm13_paths = CHM13_BAM_MAP.get(chem) or []
    combined = [p for p in hg002_paths + chm13_paths if p]
    if combined:
        BAM_MAP[chem] = combined
    else:
        BAM_MAP[chem] = None

# ── FASTQ paths (for 'fastq' mode only) ────────────────────────────
FASTQ_DIR = os.path.join(GDRIVE_DIR, 'fastq')

FASTQ_MAP = {
    'pacbio_hifi_sequel2': [
        os.path.join(FASTQ_DIR, 'SRR11292120.fastq.gz'),
        os.path.join(FASTQ_DIR, 'SRR11292121.fastq.gz'),
    ],
    'ont_lsk110_r941':     [],
    'ont_ulk001_r941': [
        os.path.join(FASTQ_DIR, 'chm13_ont_ul.fastq.gz'),
    ],
    'ont_lsk114_r1041':            [],
    'ont_r1041_duplex':            [],
    'ont_ulk114_r1041':            [],
    'ont_ulk114_r1041_hiacc':      [],
    'ont_ulk114_r1041_dorado':     [],
    'illumina_hiseq2500': [
        os.path.join(FASTQ_DIR, 'SRR1997411_1.fastq.gz'),
        os.path.join(FASTQ_DIR, 'SRR1997411_2.fastq.gz'),
    ],
    'pacbio_onso':         [],     # HG002 BAMs already aligned
    'pacbio_hifi_revio':   [],     # HG002 BAMs already aligned
    'element_aviti':       [],     # HG002 BAMs already aligned
    'element_ultraq':      [],     # HG002 BAMs already aligned
}

# ── Reference genomes ──────────────────────────────────────────────
HG002_REF_PATH = os.path.join(GDRIVE_DIR, 'ErrorSmith_bams', 'HG002', 'reference', 'hg002v1.0.1.fasta')
CHM13_REF_PATH = os.path.join(GDRIVE_DIR, 'chm13v2.0.fa')

if os.path.exists(HG002_REF_PATH) or os.path.exists(HG002_REF_PATH + '.gz'):
    REFERENCE_PATH = HG002_REF_PATH if os.path.exists(HG002_REF_PATH) else HG002_REF_PATH + '.gz'
    print(f"✓ Using HG002v1.0.1 reference (Q100)")
else:
    REFERENCE_PATH = CHM13_REF_PATH
    print(f"ℹ HG002 reference not found, using CHM13v2.0")

AUTO_DOWNLOAD_REF = True

# ── Subsampling ─────────────────────────────────────────────────────
ERRORSMITH_SUBSAMPLE = 5_000_000  # 5M bases/chemistry → ~300k error rows total

# ── Chromosomes to process ──────────────────────────────────────────
ERRORSMITH_CHROMS = [f'chr{i}' for i in range(1, 6)]

# ── Threads ─────────────────────────────────────────────────────────
ERRORSMITH_THREADS = 4

# ── Validate inputs ────────────────────────────────────────────────
from generate_errorsmith_training_data import CHEMISTRY_CODES, CHEMISTRY_PRESETS

print(f"Mode: {ERRORSMITH_MODE}\n")

if ERRORSMITH_MODE == 'bam':
    print("Chemistry-specific BAMs:")
    for chem in CHEMISTRY_CODES:
        paths = BAM_MAP.get(chem)
        if paths is None:
            print(f"  ⏳ {chem:30s}: not yet mapped (placeholder)")
        elif isinstance(paths, list):
            found = [p for p in paths if os.path.exists(p)]
            missing = [p for p in paths if not os.path.exists(p)]
            if found:
                print(f"  ✓ {chem:30s}: {len(found)} BAM(s) found")
                for f in found:
                    sz = os.path.getsize(f) / (1024**3)
                    print(f"      {os.path.basename(f)} ({sz:.1f} GB)")
            if missing:
                for f in missing:
                    print(f"  ✗ {chem:30s}: not found — {os.path.basename(f)}")

elif ERRORSMITH_MODE == 'fastq':
    print("Chemistry-specific FASTQs:")
    for chem in CHEMISTRY_CODES:
        fqs = FASTQ_MAP.get(chem, [])
        if not fqs:
            print(f"  ⏳ {chem:30s}: no FASTQs (skipping)")
            continue
        found = [f for f in fqs if os.path.exists(f)]
        missing = [f for f in fqs if not os.path.exists(f)]
        if found:
            print(f"  ✓ {chem:30s}: {len(found)} file(s) found")
        if missing:
            for f in missing:
                print(f"  ✗ {chem:30s}: not found — {os.path.basename(f)}")

print(f"\n  Subsample:    {ERRORSMITH_SUBSAMPLE:,} bases/chemistry")
print(f"  Chromosomes:  {ERRORSMITH_CHROMS}")
print(f"  Reference:    {REFERENCE_PATH}")

## 3. Download Reference + Install Tools + Align FASTQs

In [ ]:
# ── Download CHM13v2.0 reference if needed ───────────────────────────
from generate_errorsmith_training_data import download_reference, CHEMISTRY_PRESETS

if not os.path.exists(REFERENCE_PATH) and AUTO_DOWNLOAD_REF:
    print("Downloading CHM13v2.0 reference (800 MB)...")
    ref_dir = os.path.dirname(REFERENCE_PATH)
    downloaded_ref = download_reference(Path(ref_dir))
    REFERENCE_PATH = str(downloaded_ref)
    print(f"✓ Reference downloaded: {REFERENCE_PATH}")
elif os.path.exists(REFERENCE_PATH):
    print(f"✓ Reference found: {REFERENCE_PATH}")
else:
    print(f"⚠ Reference not found at {REFERENCE_PATH}")
    print(f"  Set AUTO_DOWNLOAD_REF = True to download automatically")

# ── Install minimap2 + samtools (always needed for 'fastq' mode) ────
if ERRORSMITH_MODE == 'fastq':
    print("\nInstalling minimap2, samtools...")
    !apt-get install -qq -y samtools minimap2
    !which minimap2 && echo "✓ minimap2" || echo "✗ minimap2 missing"
    !which samtools && echo "✓ samtools" || echo "✗ samtools missing"

# ── Align FASTQs → sorted BAMs (for 'fastq' mode) ──────────────────
if ERRORSMITH_MODE == 'fastq':
    import subprocess

    BAM_OUT_DIR = os.path.join(ERRORSMITH_OUT, 'bams')
    os.makedirs(BAM_OUT_DIR, exist_ok=True)

    def align_fastqs(fastq_files, chemistry, ref_path, out_dir, threads=4):
        """Align FASTQ(s) with minimap2, sort + index with samtools."""
        preset = CHEMISTRY_PRESETS[chemistry]
        bam_path = os.path.join(out_dir, f'chm13_{chemistry}.sorted.bam')
        bai_path = bam_path + '.bai'

        if os.path.exists(bam_path) and os.path.exists(bai_path):
            print(f"  ✓ {chemistry} BAM already exists: {bam_path}")
            return bam_path

        # Filter to only files that exist
        existing = [f for f in fastq_files if os.path.exists(f)]
        if not existing:
            print(f"  ✗ {chemistry}: no FASTQ files found, skipping")
            return None

        print(f"  Aligning {chemistry} ({len(existing)} file(s)) with minimap2 -x {preset}...")
        print(f"    Files: {[os.path.basename(f) for f in existing]}")

        # minimap2 | samtools sort
        mm_cmd = ['minimap2', '-ax', preset, '-t', str(threads), ref_path] + existing
        sort_cmd = ['samtools', 'sort', '-@', str(threads), '-o', bam_path]

        with subprocess.Popen(mm_cmd, stdout=subprocess.PIPE) as mm:
            subprocess.run(sort_cmd, stdin=mm.stdout, check=True)

        # Index
        subprocess.run(['samtools', 'index', '-@', str(threads), bam_path], check=True)

        # Report size
        sz = os.path.getsize(bam_path) / (1024**3)
        print(f"  ✓ {chemistry} BAM ready: {bam_path} ({sz:.1f} GB)")
        return bam_path

    # Align each chemistry that has FASTQs
    print("\n── Aligning FASTQs to CHM13v2.0 ──────────────────────────")
    for chem, fqs in FASTQ_MAP.items():
        if not fqs:
            print(f"  ⏳ {chem}: no FASTQs, skipping")
            continue
        result = align_fastqs(fqs, chem, REFERENCE_PATH, BAM_OUT_DIR, ERRORSMITH_THREADS)
        if result:
            # Update BAM_MAP so the data generation cell finds the aligned BAMs
            BAM_MAP[chem] = [result]

    # Switch mode so the data generation cell uses the new BAMs
    ERRORSMITH_MODE = 'bam'
    print("\n✓ Alignment complete — mode switched to 'bam' for data generation")

## 4. Generate ErrorSmith Training Data
Processes CHM13 BAMs, parses CIGAR alignments, extracts per-base error features.

In [ ]:
# ── Generate ErrorSmith training CSV ──────────────────────────────────
from generate_errorsmith_training_data import generate_errorsmith_training_data

# Build bam_map: chemistry → single BAM path
# For chemistries with multiple BAMs (e.g. HiFi has 2 parts),
# the training script processes each BAM under the same chemistry code.
# Here we flatten lists → pass only the first BAM per chemistry for now.
# To use ALL BAMs, merge them first or extend the script to accept lists.
resolved_bam_map = {}
for chem, paths in BAM_MAP.items():
    if paths is None:
        print(f"  ⏳ {chem}: placeholder — skipping")
        continue
    for p in paths:
        if os.path.exists(p):
            # Use chemistry + index suffix for multiple BAMs of same chemistry
            key = chem
            if key in resolved_bam_map:
                # Multiple BAMs for same chemistry — append part number
                i = 2
                while f"{chem}_part{i}" in resolved_bam_map:
                    i += 1
                key = f"{chem}_part{i}"
            resolved_bam_map[key] = p
        else:
            print(f"  ✗ {chem}: BAM not found — {os.path.basename(p)}")

es_kwargs = {
    'output_dir': Path(ERRORSMITH_OUT),
    'reference_path': REFERENCE_PATH,
    'bam_map': resolved_bam_map,
    'subsample': ERRORSMITH_SUBSAMPLE,
    'chroms': ERRORSMITH_CHROMS,
    'threads': ERRORSMITH_THREADS,
    'seed': SEED,
}

print(f"\nStarting ErrorSmith data generation...")
print(f"  Mode:        {ERRORSMITH_MODE}")
print(f"  Chemistries: {list(resolved_bam_map.keys())}")
print(f"  BAM count:   {len(resolved_bam_map)}")
print()

t0 = time.time()
errorsmith_csv = generate_errorsmith_training_data(**es_kwargs)
elapsed = time.time() - t0

print(f"\n✅ ErrorSmith training CSV: {errorsmith_csv}")
print(f"   Elapsed: {elapsed/60:.1f} min")

In [ ]:
# ── Inspect ErrorSmith data ──────────────────────────────────────────
from generate_errorsmith_training_data import CHEMISTRY_NAMES

df_es = pd.read_csv(errorsmith_csv)
print(f"Shape: {df_es.shape}")

# Class distribution
ERROR_TYPE_NAMES = {0: 'correct', 1: 'substitution', 2: 'insertion', 3: 'deletion', 4: 'homopolymer_error'}
print(f"\nClass distribution:")
for label, count in df_es['error_type'].value_counts().sort_index().items():
    pct = count / len(df_es) * 100
    name = ERROR_TYPE_NAMES.get(label, f'unknown_{label}')
    print(f"  {label} ({name:20s}): {count:>10,} ({pct:5.1f}%)")

# Chemistry breakdown
print(f"\nChemistry breakdown (technology_encoded):")
for code, count in df_es['technology_encoded'].value_counts().sort_index().items():
    chem_name = CHEMISTRY_NAMES.get(int(code), f'unknown_{code}')
    pct = count / len(df_es) * 100
    print(f"  {int(code)} ({chem_name:25s}): {count:>10,} ({pct:5.1f}%)")

if 'technology' in df_es.columns:
    print(f"\nTechnology label breakdown:")
    print(df_es['technology'].value_counts().to_string())

print(f"\nFeature summary:")
feature_cols = ['base_quality', 'mean_quality_window_5', 'position_in_read',
                'gc_content_local', 'homopolymer_length', 'read_length']
df_es[feature_cols].describe().round(3)

## 5. Train ErrorSmith Classifier
XGBoost multiclass classifier with **Optuna HP search** + **hybrid resampling** (undersample majority `correct` class, oversample minority error types).

In [ ]:
# ── ErrorSmith training with Optuna + hybrid resampling ──────────────
import optuna
import pickle
import json
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix
)
from sklearn.utils import resample
from collections import Counter

optuna.logging.set_verbosity(optuna.logging.WARNING)

# ── Chemistry binary feature decomposition ──────────────────────────
# 10 binary axes derived from Flow_Cells_and_Chemistries.md.
# Let XGBoost share signal within technology families.
CHEMISTRY_FEATURE_NAMES = [
    'is_ont', 'is_pacbio_hifi', 'is_pacbio_onso', 'is_illumina',
    'is_element', 'is_long_read', 'is_short_read', 'is_r10',
    'is_ultralong', 'is_duplex',
]
# fmt: off
CHEMISTRY_FEATURES = {
    #                        ont  hifi onso ilmn elem long shrt r10  ul   dup
    0:  [0,   1,   0,   0,   0,   1,   0,   0,   0,   0],   # pacbio_hifi_sequel2
    1:  [1,   0,   0,   0,   0,   1,   0,   0,   0,   0],   # ont_lsk110_r941
    2:  [1,   0,   0,   0,   0,   1,   0,   0,   1,   0],   # ont_ulk001_r941
    3:  [1,   0,   0,   0,   0,   1,   0,   1,   0,   0],   # ont_lsk114_r1041
    4:  [1,   0,   0,   0,   0,   1,   0,   1,   1,   0],   # ont_ulk114_r1041
    5:  [0,   0,   0,   1,   0,   0,   1,   0,   0,   0],   # illumina_hiseq2500
    6:  [0,   0,   1,   0,   0,   0,   1,   0,   0,   0],   # pacbio_onso
    7:  [0,   0,   0,   0,   1,   0,   1,   0,   0,   0],   # element_aviti
    8:  [0,   0,   0,   0,   1,   0,   1,   0,   0,   0],   # element_ultraq
    9:  [0,   1,   0,   0,   0,   1,   0,   0,   0,   0],   # pacbio_hifi_revio
    10: [1,   0,   0,   0,   0,   1,   0,   1,   0,   1],   # ont_r1041_duplex
    11: [1,   0,   0,   0,   0,   1,   0,   1,   1,   0],   # ont_ulk114_r1041_hiacc
    12: [1,   0,   0,   0,   0,   1,   0,   1,   1,   0],   # ont_ulk114_r1041_dorado
}
# fmt: on

# 26 error features (16 original + 10 chemistry binary axes)
ERRORSMITH_FEATURES = [
    'base_quality', 'mean_quality_window_5', 'mean_quality_window_20',
    'position_in_read', 'read_length',
    'gc_content_local', 'gc_content_read',
    'homopolymer_length', 'homopolymer_base', 'distance_to_hp',
    'trinucleotide_context', 'pentanucleotide_context',
    'technology_encoded',       # 0-12 granular chemistry code
    # 10 binary chemistry axes (shared within technology families)
    'is_ont', 'is_pacbio_hifi', 'is_pacbio_onso', 'is_illumina',
    'is_element', 'is_long_read', 'is_short_read', 'is_r10',
    'is_ultralong', 'is_duplex',
    # Reference context
    'ref_gc_window_50', 'ref_repeat_flag', 'ref_homopolymer_length',
]

ES_LABEL = 'error_type'
ES_N_OPTUNA_TRIALS = 25
ES_N_CV_FOLDS = 5


def hybrid_resample(X, y, max_majority=100_000, random_state=42):
    """
    Hybrid resampling: undersample majority to max_majority,
    oversample minorities to median class size.
    """
    classes, counts = np.unique(y, return_counts=True)
    class_counts = dict(zip(classes, counts))

    # Cap majority class
    capped = {c: min(n, max_majority) for c, n in class_counts.items()}
    median_size = int(np.median(list(capped.values())))

    X_resampled, y_resampled = [], []
    for cls in classes:
        mask = y == cls
        X_cls, y_cls = X[mask], y[mask]
        target = max(median_size, min(len(X_cls), max_majority))

        if len(X_cls) > target:
            X_rs, y_rs = resample(X_cls, y_cls, n_samples=target,
                                  random_state=random_state, replace=False)
        elif len(X_cls) < target:
            X_rs, y_rs = resample(X_cls, y_cls, n_samples=target,
                                  random_state=random_state, replace=True)
        else:
            X_rs, y_rs = X_cls, y_cls

        X_resampled.append(X_rs)
        y_resampled.append(y_rs)

    return np.vstack(X_resampled), np.concatenate(y_resampled)


def es_optuna_objective(trial, X, y):
    """Optuna objective for ErrorSmith multiclass classifier."""
    params = {
        'max_depth': trial.suggest_int('max_depth', 5, 14),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'tree_method': XGB_TREE,
        'device': XGB_DEVICE,
        'objective': 'multi:softprob',
        'num_class': 5,
        'eval_metric': 'mlogloss',
        'random_state': SEED,
    }

    model = xgb.XGBClassifier(**params)
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    scores = cross_val_score(model, X, y, cv=skf, scoring='f1_macro')
    return scores.mean()


# ── Prepare data ────────────────────────────────────────────────────
df_es = pd.read_csv(errorsmith_csv)

# Backward compat: derive binary features if CSV was generated before
# the chemistry feature decomposition was added.
if 'is_ont' not in df_es.columns:
    print("⚠ CSV lacks binary chemistry columns — deriving from technology_encoded")
    for idx, row in df_es.iterrows():
        code = int(row['technology_encoded'])
        vec = CHEMISTRY_FEATURES.get(code, [0]*len(CHEMISTRY_FEATURE_NAMES))
        for fname, val in zip(CHEMISTRY_FEATURE_NAMES, vec):
            df_es.at[idx, fname] = val
    # Ensure int type
    for fname in CHEMISTRY_FEATURE_NAMES:
        df_es[fname] = df_es[fname].astype(int)

X_es = df_es[ERRORSMITH_FEATURES].fillna(0).values
y_es = df_es[ES_LABEL].values

print(f"Raw data: {X_es.shape[0]:,} samples × {X_es.shape[1]} features")
print(f"Class distribution: {dict(Counter(y_es))}")

# Show which chemistries are represented
unique_chems = sorted(df_es['technology_encoded'].unique())
print(f"Chemistries present: {[CHEMISTRY_NAMES.get(int(c), f'unk_{c}') for c in unique_chems]}")

# ── Hybrid resampling ──────────────────────────────────────────────
X_balanced, y_balanced = hybrid_resample(X_es, y_es, max_majority=100_000)
print(f"\nAfter hybrid resampling: {X_balanced.shape[0]:,} samples")
print(f"Balanced distribution:  {dict(Counter(y_balanced))}")

# ── Optuna HP search ────────────────────────────────────────────────
print(f"\nRunning Optuna ({ES_N_OPTUNA_TRIALS} trials)...")
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: es_optuna_objective(trial, X_balanced, y_balanced),
               n_trials=ES_N_OPTUNA_TRIALS, show_progress_bar=True)

best_params = study.best_params
best_params['tree_method'] = XGB_TREE
best_params['device'] = XGB_DEVICE
best_params['objective'] = 'multi:softprob'
best_params['num_class'] = 5
best_params['eval_metric'] = 'mlogloss'
best_params['random_state'] = SEED

print(f"\nBest F1-macro: {study.best_value:.4f}")
print(f"Best params: depth={best_params['max_depth']}, "
      f"lr={best_params['learning_rate']:.4f}, "
      f"n_est={best_params['n_estimators']}")

# ── 5-fold CV with best params ──────────────────────────────────────
print(f"\nRunning {ES_N_CV_FOLDS}-fold CV...")
skf = StratifiedKFold(n_splits=ES_N_CV_FOLDS, shuffle=True, random_state=SEED)

cv_acc = []
cv_f1 = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_balanced, y_balanced)):
    X_train, X_val = X_balanced[train_idx], X_balanced[val_idx]
    y_train, y_val = y_balanced[train_idx], y_balanced[val_idx]

    model = xgb.XGBClassifier(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')
    cv_acc.append(acc)
    cv_f1.append(f1)
    print(f"  Fold {fold+1}: Accuracy={acc:.4f}, F1-macro={f1:.4f}")

print(f"\nCV Summary: Accuracy={np.mean(cv_acc):.4f}±{np.std(cv_acc):.4f}, "
      f"F1-macro={np.mean(cv_f1):.4f}±{np.std(cv_f1):.4f}")

# ── Final model (train on all balanced data) ────────────────────────
print(f"\nTraining final model on all {len(y_balanced):,} samples...")
final_es_model = xgb.XGBClassifier(**best_params)
final_es_model.fit(X_balanced, y_balanced)

# Save
errorsmith_save_dir = os.path.join(MODELS_OUT, 'errorsmith')
os.makedirs(errorsmith_save_dir, exist_ok=True)

pkl_path = os.path.join(errorsmith_save_dir, 'error_classifier.pkl')
with open(pkl_path, 'wb') as f:
    pickle.dump(final_es_model, f)

print(f"\n✓ Saved: {pkl_path}")

# ── Classification report on held-out fold ──────────────────────────
print(f"\n{'═'*60}")
print("Classification Report (last CV fold):")
print(f"{'═'*60}")
class_names = ['correct', 'substitution', 'insertion', 'deletion', 'HP_error']
print(classification_report(y_val, y_pred, target_names=class_names, digits=3))

# Save results
errorsmith_results = {
    'best_params': best_params,
    'cv_accuracy_mean': float(np.mean(cv_acc)),
    'cv_accuracy_std': float(np.std(cv_acc)),
    'cv_f1_macro_mean': float(np.mean(cv_f1)),
    'cv_f1_macro_std': float(np.std(cv_f1)),
    'n_samples_raw': int(len(y_es)),
    'n_samples_balanced': int(len(y_balanced)),
    'n_features': len(ERRORSMITH_FEATURES),
    'class_distribution': dict(Counter(int(c) for c in y_balanced)),
    'chemistries_used': [CHEMISTRY_NAMES.get(int(c), str(c)) for c in unique_chems],
    'reference': 'HG002v1.0.1' if 'hg002' in REFERENCE_PATH.lower() else 'CHM13v2.0',
    'feature_names': ERRORSMITH_FEATURES,
}
with open(os.path.join(errorsmith_save_dir, 'training_results.json'), 'w') as f:
    json.dump(errorsmith_results, f, indent=2, default=str)

print(f"\n{'═'*60}")
print(f"  ErrorSmith training complete!")
print(f"  Model saved to: {pkl_path}")
print(f"  Features: {len(ERRORSMITH_FEATURES)} (16 base + 10 chemistry axes)")
print(f"  Reference: {errorsmith_results['reference']}")
print(f"  Chemistries used: {[CHEMISTRY_NAMES.get(int(c), str(c)) for c in unique_chems]}")
print(f"{'═'*60}")

## 6. Feature Importance

In [ ]:
# ── Feature importance plot ──────────────────────────────────────────
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
fig.suptitle('Feature Importance — ErrorSmith Error Classifier', fontsize=16, fontweight='bold')

es_importances = final_es_model.feature_importances_
sorted_idx = np.argsort(es_importances)[-16:]  # All 16 features
ax.barh(range(len(sorted_idx)), es_importances[sorted_idx], color='coral')
ax.set_yticks(range(len(sorted_idx)))
ax.set_yticklabels([ERRORSMITH_FEATURES[i] for i in sorted_idx], fontsize=10)
ax.set_xlabel('Importance')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(os.path.join(MODELS_OUT, 'errorsmith_feature_importance.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✓ Feature importance plot saved")

## 7. Export Models to Google Drive

In [ ]:
# ── Package and export to Google Drive ────────────────────────────────
import tarfile, shutil, glob

GDRIVE_TARBALL = os.path.join(GDRIVE_DIR, 'errorsmith_models.tar.gz')

# List all model files
model_files = glob.glob(f'{errorsmith_save_dir}/**/*', recursive=True)
print(f"Model files to export ({len(model_files)}):")
for f in sorted(model_files):
    if os.path.isfile(f):
        size_kb = os.path.getsize(f) / 1024
        print(f"  {os.path.relpath(f, errorsmith_save_dir):40s} {size_kb:>8.1f} KB")

# Create tarball
print(f"\nCreating tarball...")
with tarfile.open(GDRIVE_TARBALL, 'w:gz') as tar:
    tar.add(errorsmith_save_dir, arcname='errorsmith_models')

tarball_size = os.path.getsize(GDRIVE_TARBALL) / (1024 * 1024)
print(f"\n✅ Exported to Google Drive:")
print(f"   {GDRIVE_TARBALL}")
print(f"   Size: {tarball_size:.1f} MB")

# Also copy to Drive for easy access
if os.path.exists(GDRIVE_ERRORSMITH):
    shutil.rmtree(GDRIVE_ERRORSMITH)
shutil.copytree(errorsmith_save_dir, GDRIVE_ERRORSMITH)
print(f"   Copied → {GDRIVE_ERRORSMITH}")

print(f"\n📥 To download locally:")
print(f"   1. Open Google Drive → My Drive/Colab Notebooks/strandweaver/")
print(f"   2. Download errorsmith_models.tar.gz")
print(f"   3. Extract to strandweaver/trained_models/errorsmith/")

## 8. Summary

In [ ]:
# ── Final summary ────────────────────────────────────────────────────
chems_used = errorsmith_results.get('chemistries_used', [])
ref_used = errorsmith_results.get('reference', 'unknown')
print("╔══════════════════════════════════════════════════════════════════╗")
print("║         ErrorSmith Model Training — Summary                     ║")
print("╠══════════════════════════════════════════════════════════════════╣")
print("║                                                                  ║")
print("║  1 XGBoost Multiclass Classifier:                                ║")
print("║  ────────────────────────────────                                 ║")
print(f"║    error_classifier       F1={errorsmith_results['cv_f1_macro_mean']:.3f}±"
      f"{errorsmith_results['cv_f1_macro_std']:.3f}  "
      f"Acc={errorsmith_results['cv_accuracy_mean']:.3f}    ║")
print("║                                                                  ║")
print("║  Classes: correct | substitution | insertion |                   ║")
print("║           deletion | homopolymer_error                           ║")
print("║                                                                  ║")
print(f"║  Reference:    {ref_used:40s}           ║")
print(f"║  Chemistries trained ({len(chems_used)}):                       ║")
for c in chems_used:
    print(f"║    • {c:55s}       ║")
print("║                                                                  ║")
print("║  Chemistry taxonomy (13 codes):                                  ║")
print("║     0: pacbio_hifi_sequel2       (HiFi Sequel II/IIe)            ║")
print("║     1: ont_lsk110_r941           (Ligation, R9.4.1)              ║")
print("║     2: ont_ulk001_r941           (Ultra-long, R9.4.1)            ║")
print("║     3: ont_lsk114_r1041          (Ligation, R10.4.1)             ║")
print("║     4: ont_ulk114_r1041          (Ultra-long, R10.4.1)           ║")
print("║     5: illumina_hiseq2500        (HiSeq 2500)                   ║")
print("║     6: pacbio_onso               (Onso short-read SBB)           ║")
print("║     7: element_aviti             (Aviti standard/long)           ║")
print("║     8: element_ultraq            (UltraQ)                        ║")
print("║     9: pacbio_hifi_revio         (HiFi Revio)                    ║")
print("║    10: ont_r1041_duplex          (R10.4.1 Duplex)                ║")
print("║    11: ont_ulk114_r1041_hiacc    (UL R10.4.1 HiAcc exp.)        ║")
print("║    12: ont_ulk114_r1041_dorado   (UL R10.4.1 Dorado)            ║")
print("║                                                                  ║")
print("║  Output files on Google Drive:                                   ║")
print(f"║    {GDRIVE_DIR}/")
print(f"║    ├── errorsmith_models/    (1 .pkl + results JSON)")
print(f"║    └── errorsmith_models.tar.gz")
print("║                                                                  ║")
print("║  Next steps:                                                     ║")
print("║    1. Download models to local strandweaver/trained_models/      ║")
print("║    2. Ensure K-Weaver models are also in place                   ║")
print("║    3. Run: strandweaver core-assemble --use-ai <reads>           ║")
print("║    4. All 7/7 AI modules now have trained weights! 🎉            ║")
print("║                                                                  ║")
print("╚══════════════════════════════════════════════════════════════════╝")